# Do not look at part 2, it is very dirty !!

In [3]:
with open("puzzles/10", "r") as fp:
    data = fp.read()[:-1]

In [4]:
lines = data.split("\n")

In [5]:
def process_line(line):
    i = line.find("]")
    j = line.find("{")
    
    part1 = line[1:i]
    part2 = line[i+2:j-1]
    part3 = line[j+1:-1]

    # Conver to bolean
    #part1 = [x == "#" for x in part1]

    subs = part2.split(" ")
    lst_button = []
    for sub in subs:
        lst_button.append(list(map(int, sub[1:-1].split(","))))
        

    part3 = list(map(int, part3.split(",")))
    
    return part1, lst_button, part3
    

# Part 1

In [24]:
def activate(pattern, button):
    part1 = [x == "#" for x in pattern]
    for b in button:
        part1[b] = part1[b] == False

    return "".join(["#" if x else "." for x in part1])
    

In [30]:
def solve_pattern(line):
    p1, buttons, _ = process_line(line)
    #print(p1)
    # Max number of pattern = max number of buttons
    lmax = len(buttons) 
    solution = "."*len(p1)
    lst_patterns = [p1]
    for d in range(lmax):
        lst_new = []
        for but in buttons:
            for pat in lst_patterns:
                #print(pat, but)
                pat_new = activate(pat, but)
                #print("=>", pat_new)
                lst_new.append(pat_new)

        lst_patterns = list(set(lst_new))
        if any(map(lambda x: x==solution, lst_patterns)):
            return d+1
    
        

In [34]:
tot = 0
for line in lines:
    tot += solve_pattern(line)

In [35]:
print("Part 1:", tot)

Part 1: 488


# Part 2

Need a numerical solver. Recursion will be infeasible.


For each button, iterate until patterns infeasible.

Keep track of the number of press. Stop if not minimal



## Too slow ...

In [37]:
def activate_v2(joltage, button):
    joltage_new = [x for x in joltage]
    for b in button:
        joltage_new[b] += 1
    return joltage_new

In [39]:
def check_pattern(reference, values):
    """Check if the current joltage is OK or above expected
    """
    sign = "="
    for x0, x in zip(reference, values):
        if x0 < x:
            return "too_high"
        elif x0 > x:
            sign = "<"

    if sign == "=":
        return "="
    else:
        return "low"

In [43]:
def reduce_list(lst_patterns):
    lst_tmp = []
    for pat in lst_patterns:
        lst_tmp.append(",".join(list(map(str, pat))))
    lst_tmp = list(set(lst_tmp))
    
    # Decompress
    lst_next = []
    for pat in lst_tmp:
        lst_next.append(list(map(int, pat.split(","))))

    return lst_next
        
        
    

In [52]:
def solve_pattern_p2(line):
    _, buttons, ref_joltage = process_line(line)

    # Max number of pattern = max number of buttons
    lst_patterns = [[0 for _ in range(len(ref_joltage))]]
    cnt = 0
    while True:
        cnt += 1
        lst_new = []
        for but in buttons:
            for pat in lst_patterns:
                pat_new = activate_v2(pat, but)
                lst_new.append(pat_new)


        
        lst_patterns = []
        for pat in reduce_list(lst_new):
            choice = check_pattern(ref_joltage, pat)
            if choice == "too_high":
                # Discard the element
                continue
            elif choice == "low":
                lst_patterns.append(pat)
            else: # "="
                # STOOOP
                return cnt
                
        if len(lst_patterns) == 0:
            print("ERROR")
            break
                

## Too slow p2 ...

In [57]:
def add_to_pattern(pattern, button, c):
    """
    pattern: current joltage
    button: way to increment
    c: multiplicative factor
    """
    new_pattern = [x for x in pattern]
    for b in button:
        new_pattern[b] += c

    return new_pattern

In [39]:
def check_pattern(reference, values):
    """Check if the current joltage is OK or above expected
    """
    sign = "="
    for x0, x in zip(reference, values):
        if x0 < x:
            return "too_high"
        elif x0 > x:
            sign = "<"

    if sign == "=":
        return "="
    else:
        return "low"

In [54]:
line = lines[0]
_, buttons, ref_joltage = process_line(line)


In [58]:
l = len(ref_joltage)
lst_patterns = [(0, [0 for _ in range(l)])]

# Loop over buttons
but = buttons[0]
lst_new = []
lst_final = []
cnt = 0
while True:
    cnt += 1
    flag = False
    for d, pat in lst_patterns:
        new_pat = add_to_pattern(pat, but, cnt)
        decision = check_pattern(ref_joltage, new_pat)
        if decision == "low":
            lst_new.append((d+cnt, new_pat))
            flag = True
        elif decision == "=":
            lst_final.append(d+cnt)

    if flag == False:
        break
        
        

In [61]:
len(lst_new)

37

In [62]:
l = len(ref_joltage)
lst_patterns = [(0, [0 for _ in range(l)])]

lst_final = []

# Loop over buttons
for but in buttons:
    print("Button: {} \t=> {} patterns. {} completed".format(but, len(lst_patterns), len(lst_final)))
    lst_new = []
    cnt = 0
    while True:
        cnt += 1
        flag = False
        for d, pat in lst_patterns:
            new_pat = add_to_pattern(pat, but, cnt)
            decision = check_pattern(ref_joltage, new_pat)
            if decision == "low":
                lst_new.append((d+cnt, new_pat))
                flag = True
            elif decision == "=":
                lst_final.append(d+cnt)
    
        if flag == False:
            break
            
    lst_patterns.extend(lst_new)
            
        

Button: [0, 2, 3, 6] 	=> 1 patterns. 0 completed
Button: [0, 1, 4, 6] 	=> 38 patterns. 0 completed
Button: [1, 3, 4, 5] 	=> 650 patterns. 0 completed
Button: [1, 2, 4, 6] 	=> 9750 patterns. 0 completed
Button: [0, 2, 3, 4, 5] 	=> 93600 patterns. 0 completed
Button: [2, 3, 6] 	=> 1426269 patterns. 0 completed


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x74390fcedf10>>
Traceback (most recent call last):
  File "/home/japoneris/Code/main_env/HMPV/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [65]:
ref_joltage

[37, 24, 84, 71, 44, 32, 71]

In [56]:
print(ref_joltage)
buttons


[37, 24, 84, 71, 44, 32, 71]


[[0, 2, 3, 6],
 [0, 1, 4, 6],
 [1, 3, 4, 5],
 [1, 2, 4, 6],
 [0, 2, 3, 4, 5],
 [2, 3, 6],
 [1, 2],
 [2, 3, 4, 5, 6]]

## Equation Solver ...

In [6]:
def format_system(ref_joltage, buttons):
    lst_eq = [] # list equation equal to 0
    for idx, v in enumerate(ref_joltage):
        dico = {}
        for jdx, button in enumerate(buttons):
            if idx in button:
                dico["b{}".format(jdx)] = 1

        dico["sum"] = -v
        lst_eq.append(dico)
    return lst_eq
        
        

In [110]:
def simplify_equations(lst_eqs):
    dico = lst_eqs[0]
    
    # Coeff, button
    button_keys = list(filter(lambda x: x.startswith("b"), dico.keys()))

    # Try to express this variable as the function of the other
    b0 = button_keys[0]
    c0 = dico[b0]

    dic_exp = {}
    for k, v in dico.items():
        if k == b0:
            continue
        else:
            dic_exp[k] = -v/c0

    # Then, replace in all other equations
    for dic1 in lst_eqs[1:]:
        if b0 in dic1:
            c = dic1[b0]
            del(dic1[b0])
            for k, v in dic_exp.items():
                if k not in dic1:
                    dic1[k] = v*c
                else:
                    dic1[k] = dic1[k] + v*c
                    if (dic1[k] == 0):
                        del(dic1[k])

    lst_next = []
    for dic in lst_eqs[1:]:
        if "sum" in dic:
            if len(dic) > 1:
                lst_next.append(dic)
            # else: no
        elif len(dic) > 0:
            lst_next.append(dic)
            
    
    
    return lst_next, (b0, dic_exp)

## Hand doing

### Try to complete

In [13]:
def solve_equation(lst_vals, n=0):
    """
    :param n: number of variable to look for (> than then number of vals)
    """
    dic_values = {}
    for c, dic in lst_vals[::-1]:
        if (len(dic) == 1) & ("sum" in dic):
            # Happy case
            dic_values[c] = dic["sum"]
            
            # Replace in all other values
            for _, item in lst_vals:
                if c in item:
                    item["sum"] = item.get("sum", 0) + item[c] * dic["sum"]
                    del(item[c])
                    
    return dic_values

In [18]:
def compare_solution(lst_sol, ref_joltage, buttons):
    tmp = [0 for _ in ref_joltage]
    for x, v in lst_sol.items():
        vi = int(round(v))
        i = int(x[1:])
        for b in buttons[i]:
            tmp[b] += vi

    flag = "equal"
    for x, y in zip(ref_joltage, tmp):
        if x < y:
            return "high"
        elif x > y:
            flag = "low"
    
    return flag 
    

In [34]:
def verify_solution(lst_sol, ref_joltage, buttons, verbose=True):
    tmp = [0 for _ in ref_joltage]
    for x, v in lst_sol.items():
        vi = int(round(v))
        i = int(x[1:])
        for b in buttons[i]:
            tmp[b] += vi
    
    if verbose:
        print("truth", ref_joltage)
        print("resul", tmp)
        print()

    return all([x == y for x, y in zip(ref_joltage, tmp)])
    

In [25]:
def buttons_to_joltage(buttons, dico, ref_joltage):
    current = [0 for _ in ref_joltage]
    for bid, v in dico.items():
        ID = int(bid[1:])
        for i in buttons[ID]:
            current[i] += v

    return all([x <= y for x, y in zip(current, ref_joltage)])


In [28]:
def generate_valid_patterns(buttons, ref_joltage, IDs):
    """Generate valid dictionnary to test for the provided IDs
    """
    lst_sols = [dict([('b{}'.format(ID), 0) for ID in IDs])]
    for ID in IDs:
        bid = "b{}".format(ID)
        button = buttons[ID]
        cnt = 1
        lst_previous = lst_sols
        
        while len(lst_previous) > 0:
            lst_new = []
            for item in lst_previous:
                item1 = item.copy()
                item1[bid] = item1.get(bid, 0) + 1
                tf = buttons_to_joltage(buttons, item1, ref_joltage)
                if tf:
                    lst_new.append(item1)
            
            lst_sols.extend(lst_new)
            lst_previous = lst_new

    return lst_sols

            

### Runn example

In [111]:

line = lines[128]
_, buttons, ref_joltage = process_line(line)
print(len(ref_joltage), len(buttons))
print(ref_joltage)
print(buttons)

lst_eqs = format_system(ref_joltage, buttons)
print(lst_eqs)
# Reduce equations until there is no free variables
lst_ref = []
while len(lst_eqs) > 0:
    print(lst_eqs)
    lst_eqs, resp = simplify_equations(lst_eqs)
    lst_ref.append(resp)


8 9
[21, 24, 33, 36, 48, 5, 62, 38]
[[2, 3, 6], [0, 1, 4, 7], [2, 4, 6, 7], [2, 4, 6], [0, 3, 6], [1, 4, 6, 7], [0, 1, 3, 4, 6, 7], [3, 4, 5, 6], [0, 3, 4, 6]]
[{'b1': 1, 'b4': 1, 'b6': 1, 'b8': 1, 'sum': -21}, {'b1': 1, 'b5': 1, 'b6': 1, 'sum': -24}, {'b0': 1, 'b2': 1, 'b3': 1, 'sum': -33}, {'b0': 1, 'b4': 1, 'b6': 1, 'b7': 1, 'b8': 1, 'sum': -36}, {'b1': 1, 'b2': 1, 'b3': 1, 'b5': 1, 'b6': 1, 'b7': 1, 'b8': 1, 'sum': -48}, {'b7': 1, 'sum': -5}, {'b0': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1, 'b7': 1, 'b8': 1, 'sum': -62}, {'b1': 1, 'b2': 1, 'b5': 1, 'b6': 1, 'sum': -38}]
[{'b1': 1, 'b4': 1, 'b6': 1, 'b8': 1, 'sum': -21}, {'b1': 1, 'b5': 1, 'b6': 1, 'sum': -24}, {'b0': 1, 'b2': 1, 'b3': 1, 'sum': -33}, {'b0': 1, 'b4': 1, 'b6': 1, 'b7': 1, 'b8': 1, 'sum': -36}, {'b1': 1, 'b2': 1, 'b3': 1, 'b5': 1, 'b6': 1, 'b7': 1, 'b8': 1, 'sum': -48}, {'b7': 1, 'sum': -5}, {'b0': 1, 'b2': 1, 'b3': 1, 'b4': 1, 'b5': 1, 'b6': 1, 'b7': 1, 'b8': 1, 'sum': -62}, {'b1': 1, 'b2': 1, 'b5': 1, 'b6': 1,

In [113]:
if len(lst_ref) == len(buttons):
    lst_sol = solve_equation(lst_ref)
else:
    n = len(buttons)
    print("Need help to solve")
    
    S = set(["b{}".format(i) for i in range(n)])- set([x[0] for x in lst_ref])
    print(S, "missing")
    
    lst_valid = []
    lst_patterns = generate_valid_patterns(buttons, ref_joltage, [int(bid[1:]) for bid in S])
    for pat in lst_patterns:
        lst_tmp = []
        for bid, dic in lst_ref:
            lst_tmp.append((bid, dic.copy()))
        for bid, v in pat.items():
            lst_tmp.append((bid, {"sum": v}))

        
        eqq = solve_equation(lst_tmp)
        eqq = dict([(x, int(round(y))) for x, y in eqq.items()])
        print(eqq)
        tf = verify_solution(eqq, ref_joltage, buttons, True)
        if tf:
            # Filter valid values
            if all([v >= 0 for v in eqq.values()]):
                # Filter negative values
                lst_valid.append(eqq)
    
    

Need help to solve
{'b8'} missing
{'b8': 0, 'b3': 5, 'b6': 13, 'b7': 5, 'b2': 14, 'b4': 4, 'b0': 14, 'b5': 7, 'b1': 4}
truth [21, 24, 33, 36, 48, 5, 62, 38]
resul [21, 24, 33, 36, 48, 5, 62, 38]

{'b8': 1, 'b3': 4, 'b6': 11, 'b7': 5, 'b2': 14, 'b4': 4, 'b0': 15, 'b5': 8, 'b1': 5}
truth [21, 24, 33, 36, 48, 5, 62, 38]
resul [21, 24, 33, 36, 48, 5, 62, 38]

{'b8': 2, 'b3': 3, 'b6': 9, 'b7': 5, 'b2': 14, 'b4': 4, 'b0': 16, 'b5': 9, 'b1': 6}
truth [21, 24, 33, 36, 48, 5, 62, 38]
resul [21, 24, 33, 36, 48, 5, 62, 38]

{'b8': 3, 'b3': 2, 'b6': 7, 'b7': 5, 'b2': 14, 'b4': 4, 'b0': 17, 'b5': 10, 'b1': 7}
truth [21, 24, 33, 36, 48, 5, 62, 38]
resul [21, 24, 33, 36, 48, 5, 62, 38]

{'b8': 4, 'b3': 1, 'b6': 5, 'b7': 5, 'b2': 14, 'b4': 4, 'b0': 18, 'b5': 11, 'b1': 8}
truth [21, 24, 33, 36, 48, 5, 62, 38]
resul [21, 24, 33, 36, 48, 5, 62, 38]

{'b8': 5, 'b3': 0, 'b6': 3, 'b7': 5, 'b2': 14, 'b4': 4, 'b0': 19, 'b5': 12, 'b1': 9}
truth [21, 24, 33, 36, 48, 5, 62, 38]
resul [21, 24, 33, 36, 48, 5, 62, 

## Full loop

In [114]:
len(lst_tot), len(lines)

(129, 177)

In [115]:
lst_tot = []
for idx, line in enumerate(lines):
    _, buttons, ref_joltage = process_line(line)
    lst_eqs = format_system(ref_joltage, buttons)

    # Reduce equations until there is no free variables
    lst_ref = []
    while len(lst_eqs) > 0:
        lst_eqs, resp = simplify_equations(lst_eqs)
        lst_ref.append(resp)


    lst_ref = list(filter(lambda x: len(x[1]) > 0, lst_ref))
    
    if len(lst_ref) == len(buttons):
        lst_sol = solve_equation(lst_ref)
        lst_tot.append((idx, sum(lst_sol.values())))
        
    else:
        n = len(buttons)
        print(idx, "Need help to solve")
        
        S = set(["b{}".format(i) for i in range(n)])- set([x[0] for x in lst_ref])
        print("\t", S, "missing")
        
        lst_valid = []
        lst_patterns = generate_valid_patterns(buttons, ref_joltage, [int(bid[1:]) for bid in S])
        for pat in lst_patterns:
            lst_tmp = []
            for bid, dic in lst_ref:
                lst_tmp.append((bid, dic.copy()))
            for bid, v in pat.items():
                lst_tmp.append((bid, {"sum": v}))

            eqq = solve_equation(lst_tmp)
            eqq = dict([(x, int(round(y))) for x, y in eqq.items()])
            tf = verify_solution(eqq, ref_joltage, buttons, False)
            if tf:
                # Filter valid values
                if all([v >= 0 for v in eqq.values()]):
                    # Filter negative values
                    lst_valid.append(eqq)
                    
        lst_tot.append((idx, min(list(map(lambda x: sum(x.values()), lst_valid)))))
        

0 Need help to solve
	 {'b6'} missing
6 Need help to solve
	 {'b3'} missing
9 Need help to solve
	 {'b7', 'b8'} missing
12 Need help to solve
	 {'b1'} missing
15 Need help to solve
	 {'b7'} missing
16 Need help to solve
	 {'b3'} missing
17 Need help to solve
	 {'b7'} missing
20 Need help to solve
	 {'b5'} missing
23 Need help to solve
	 {'b7', 'b5'} missing
24 Need help to solve
	 {'b6', 'b5'} missing
25 Need help to solve
	 {'b4', 'b5'} missing
27 Need help to solve
	 {'b7', 'b8'} missing
28 Need help to solve
	 {'b7', 'b9'} missing
30 Need help to solve
	 {'b6', 'b4'} missing
31 Need help to solve
	 {'b7', 'b9'} missing
32 Need help to solve
	 {'b1', 'b5'} missing
33 Need help to solve
	 {'b8', 'b4'} missing
34 Need help to solve
	 {'b3', 'b9'} missing
39 Need help to solve
	 {'b6', 'b5'} missing
41 Need help to solve
	 {'b6', 'b4'} missing
42 Need help to solve
	 {'b4'} missing
44 Need help to solve
	 {'b6', 'b5'} missing
46 Need help to solve
	 {'b6', 'b8'} missing
47 Need help to 

In [117]:
print('Part 2:', sum(map(lambda x: x[1], lst_tot)))

Part 2: 18771.0
